In [29]:
!rm -rf SAIRA && git clone https://github.com/kilimanj4r0/SAIRA.git

Cloning into 'SAIRA'...
remote: Enumerating objects: 1043, done.
remote: Counting objects: 100% (1043/1043), done.
remote: Compressing objects: 100% (767/767), done.
remote: Total 1043 (delta 306), reused 994 (delta 257), pack-reused 0
Receiving objects: 100% (1043/1043), 2.06 MiB | 5.72 MiB/s, done.
Resolving deltas: 100% (306/306), done.


In [ ]:
!pip install -q evaluate
!pip install -q rouge_score
!pip install -q bert_score

In [8]:
!pip uninstall -y preprocessing
!pip install -q -U nltk

Found existing installation: preprocessing 0.1.13
Uninstalling preprocessing-0.1.13:
  Successfully uninstalled preprocessing-0.1.13


In [1]:
import os
import json
from collections import defaultdict

import nltk

nltk.download('wordnet')
# !cd /usr/share/nltk_data/corpora/ && unzip -o wordnet.zip

from evaluate import load

/root/miniconda3/envs/vladimir-torch/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


[2023-11-27 16:21:47,567] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
!tree /kaggle/working/SAIRA/data/results

/kaggle/working/SAIRA/data/results [error opening dir]

0 directories, 0 files


In [2]:
!tree ../data/results

../data/results
├── chromadb-llama2-13b-emb-base-default
│   ├── extra_questions.json
│   ├── faq_academic.json
│   ├── faq_admin.json
│   ├── faq_extracurricular.json
│   └── faq_household.json
├── chromadb-llama2-13b-emb-base-summarize
│   ├── extra_questions.json
│   ├── faq_academic.json
│   ├── faq_admin.json
│   ├── faq_extracurricular.json
│   └── faq_household.json
├── chromadb-mistral-instruct-7b-emb-base-default
│   ├── extra_questions.json
│   ├── faq_academic.json
│   ├── faq_admin.json
│   ├── faq_extracurricular.json
│   └── faq_household.json
├── chromadb-mistral-instruct-7b-emb-base-summarize
│   ├── extra_questions.json
│   ├── faq_academic.json
│   ├── faq_admin.json
│   ├── faq_extracurricular.json
│   └── faq_household.json
├── chromadb-orca2-13b-emb-base-default
│   ├── extra_questions.json
│   ├── faq_academic.json
│   ├── faq_admin.json
│   ├── faq_extracurricular.json
│   └── faq_household.json
├── chromadb-orca2-13b-emb-base-summarize
│   ├── extra_questions.js

In [3]:
bleu_metric = load('bleu')
rouge_metric = load('rouge')
meteor_metric = load('meteor')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [4]:
# base_directory = '/kaggle/working/SAIRA/data/results'
base_directory = '../data/results'
models = os.listdir(base_directory)

# Dictionary to store scores for each model
model_scores = defaultdict(list)

for model in models:
    model_directory = os.path.join(base_directory, model)
    if os.path.isdir(model_directory):
        outputs = os.listdir(model_directory)
        
        # Temporary lists to store scores for each output of the model
        bleu_scores = []
        rouge_scores = []
        meteor_scores = []
        
        for output in outputs:
            output_file = os.path.join(model_directory, output)
            if os.path.isfile(output_file) and output.endswith('.json'):
                with open(output_file, 'r') as file:
                    data = json.load(file)
                    
                    questions = [item['question'] for item in data]
                    references = [item['answer'] for item in data]
                    predictions = [item['generated_answer'] for item in data]
                    
                    # Compute scores for the current output
                    bleu_score = bleu_metric.compute(predictions=predictions, references=references)
                    bleu_scores.append(bleu_score['bleu'])
                    
                    rouge_score = rouge_metric.compute(predictions=predictions, references=references)
                    rouge_scores.append(rouge_score['rougeL'])
                    
                    meteor_score = meteor_metric.compute(predictions=predictions, references=references)
                    meteor_scores.append(meteor_score['meteor'])
        
        # Calculate average scores for the model
        model_scores[model] = {
            'BLEU': sum(bleu_scores) / len(bleu_scores),
            'ROUGE': sum(rouge_scores) / len(rouge_scores),
            'METEOR': sum(meteor_scores) / len(meteor_scores)
        }

# Output average scores for each model
for model, scores in model_scores.items():
    print(f"Model: {model}")
    print(f"Average BLEU score:   {scores['BLEU']:.4f}")
    print(f"Average ROUGE score:  {scores['ROUGE']:.4f}")
    print(f"Average METEOR score: {scores['METEOR']:.4f}")
    print()

Model: chromadb-mistral-instruct-7b-emb-base-summarize
Average BLEU score:   0.0222
Average ROUGE score:  0.1295
Average METEOR score: 0.2212

Model: simple-llama2-13b-emb-large-default
Average BLEU score:   0.1091
Average ROUGE score:  0.3076
Average METEOR score: 0.4705

Model: chromadb-llama2-13b-emb-base-default
Average BLEU score:   0.0371
Average ROUGE score:  0.1515
Average METEOR score: 0.2774

Model: faiss-mistral-7b-emb-large-summarize
Average BLEU score:   0.0445
Average ROUGE score:  0.1913
Average METEOR score: 0.3063

Model: chromadb-orca2-13b-emb-base-summarize
Average BLEU score:   0.0195
Average ROUGE score:  0.1016
Average METEOR score: 0.2199

Model: faiss-mistral-7b-emb-large-default
Average BLEU score:   0.1619
Average ROUGE score:  0.3865
Average METEOR score: 0.4994

Model: chromadb-llama2-13b-emb-base-summarize
Average BLEU score:   0.0211
Average ROUGE score:  0.1117
Average METEOR score: 0.2255

Model: simple-mistral-instruct-7b-emb-large-summarize
Average BLE